In [1]:

import os, shutil
from glob import glob
import numpy as np
import pandas as pd
from nilearn import image, maskers
# from nilearn.maskers import NiftiMasker
from natsort import natsorted
import seaborn as sns
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from collections import defaultdict


In [2]:
# change the base dir if working on other platform:
base_dir = "/Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/"

img_dir = base_dir + "3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/"
out_dir = "out01_tms_induced_respose/"
out_file_name = "HRF_tms_induced_response.csv"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    

In [3]:
# list all images of beta values and save information to csv file:

subject = []
site = []
path = []
group = []

roi_list = ['L_Fp', 'L_aMFG', 'L_pMFG', 'R_FEF', 'R_Fp', 'R_IFJ', 'R_IPL','R_M1', 'R_aMFG', 'R_pMFG', 'R_preSMA']

tms_response_maps = []
for roi in roi_list:
    tms_response_maps += glob(img_dir + roi + "/NTHC_2ndlevel/CausCon*.nii")
    tms_response_maps += glob(img_dir + roi + "/TEHC_2ndlevel/CausCon*.nii")

tms_response_maps = natsorted(tms_response_maps)

for m in tms_response_maps:
    file_name_list = os.path.basename(m).split("_")
    subject.append(file_name_list[1][4:])
    site.append(file_name_list[3] + '_' + file_name_list[4])
    group.append(file_name_list[1][:4])
    path.append(m)
    
complete_df = pd.DataFrame()
complete_df['inputfile'] = path
complete_df['subject'] = subject
complete_df['site'] = site
complete_df['group'] = group

# complete_df.to_csv(out_dir + out_file_name, index=False)
complete_df

,inputfile,subject,site,group
0,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1001,L_Fp,NTHC
1,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1003,L_Fp,NTHC
2,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1036,L_Fp,NTHC
3,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1037,L_Fp,NTHC
4,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1038,L_Fp,NTHC
...,...,...,...,...
793,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2090,R_preSMA,TEHC
794,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2093,R_preSMA,TEHC
795,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2102,R_preSMA,TEHC
796,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2105,R_preSMA,TEHC


In [4]:
complete_df = pd.read_csv(out_dir + out_file_name)
complete_df.iloc[1]['inputfile']

# test on first 10 rows:
# complete_df = complete_df.loc[:10,:]

'/Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1003_tms_L_Fp_con_0001_mni.nii'

# TMS Site Response from TMS Induced Response Maps

In [5]:
# standard roi:
roi_dir = base_dir + "3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Sept_8/TMS_site_rois/"

tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site']
    #subject = complete_df.iloc[i]['subject']
    
    nifti_masker = maskers.NiftiMasker(glob(roi_dir + site + ".nii*")[0]).fit()
    tms_site_response.append(np.mean(nifti_masker.transform(image_name)))


complete_df['tms_site_response_standard'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)

100%|██████████| 798/798 [14:02<00:00,  1.06s/it]


In [6]:
# individual roi 6mm:
roi_dir = base_dir + "/3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Nov_3/tms_spheres/"

## error occurs for this mask: '/L-pMFG/2040_6mm.nii.gz':
## The mask is invalid as it is empty: it masks all data.
## try another roi_dir:

# roi_dir = "/Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/" \
#         + "Jing_Drive/3_Data_Working/CausalConnectome/post_processing/VisorT1_iso2nii-selected/wMNI/IndEnvMask/"

## and get the same error, skip the empty masks with try...except...

tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site'].replace('_', '-')
#     site = complete_df.iloc[i]['site']
    subject = complete_df.iloc[i]['subject']
    mask_file = glob(roi_dir + site + "/" + str(subject) + "_6mm.nii*")
        
    if mask_file:
        try:
            nifti_masker = maskers.NiftiMasker(mask_file[0]).fit()
            tms_site_response.append(np.mean(nifti_masker.transform(image_name)))
        except:
            tms_site_response.append(np.nan)
            print("error occurs for mask: ", mask_file[0])
    else:
        tms_site_response.append(np.nan)
        print("mask not found: ", image_name)

complete_df['tms_site_response_individual_6mm'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)

  0%|          | 3/798 [00:00<03:02,  4.35it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1037_tms_L_Fp_con_0001_mni.nii


  1%|          | 7/798 [00:01<02:46,  4.74it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1043_tms_L_Fp_con_0001_mni.nii


  1%|▏         | 10/798 [00:01<02:36,  5.03it/s]

...

 99%|█████████▉| 789/798 [12:31<00:01,  6.37it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2064_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2066_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2069_tms_R_preSMA_con_0001_mni.nii


 99%|█████████▉| 792/798 [12:32<00:01,  5.40it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2088_tms_R_preSMA_con_0001_mni.nii


100%|██████████| 798/798 [12:33<00:00,  1.06it/s]


In [7]:
# individual roi 10mm:
roi_dir = base_dir + "/3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Nov_3/tms_spheres/"

tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site'].replace('_', '-')
    subject = complete_df.iloc[i]['subject']
    mask_file = glob(roi_dir + site + "/" + str(subject) + "_10mm.nii*")
    
    if mask_file:
        try:
            nifti_masker = maskers.NiftiMasker(mask_file[0]).fit()
            tms_site_response.append(np.mean(nifti_masker.transform(image_name)))
        except:
            tms_site_response.append(np.nan)
            print("error occurs for mask: ", mask_file[0])
    else:
        tms_site_response.append(np.nan)
        print("mask not found: ", image_name)

complete_df['tms_site_response_individual_10mm'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)

  0%|          | 3/798 [00:00<02:56,  4.49it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1037_tms_L_Fp_con_0001_mni.nii


  1%|          | 7/798 [00:01<02:48,  4.69it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1043_tms_L_Fp_con_0001_mni.nii


  1%|▏         | 10/798 [00:01<02:40,  4.91it/s]

...

 99%|█████████▉| 789/798 [02:59<00:01,  6.18it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2064_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2066_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2069_tms_R_preSMA_con_0001_mni.nii


 99%|█████████▉| 792/798 [03:00<00:01,  5.37it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2088_tms_R_preSMA_con_0001_mni.nii


100%|██████████| 798/798 [03:01<00:00,  4.40it/s]


In [8]:
# individual roi 14mm:

roi_dir = base_dir + "/3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Nov_3/tms_spheres/"

tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site'].replace('_', '-')
    subject = complete_df.iloc[i]['subject']
    mask_file = glob(roi_dir + site + "/" + str(subject) + "_14mm.nii*")
    
    if mask_file:
        try:
            nifti_masker = maskers.NiftiMasker(mask_file[0]).fit()
            tms_site_response.append(np.mean(nifti_masker.transform(image_name)))
        except:
            tms_site_response.append(np.nan)
            print("error occurs for mask: ", mask_file[0])
    else:
        tms_site_response.append(np.nan)
        print("mask not found: ", image_name)

complete_df['tms_site_response_individual_14mm'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)


  0%|          | 3/798 [00:00<02:53,  4.59it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1037_tms_L_Fp_con_0001_mni.nii


  1%|          | 9/798 [00:01<02:17,  5.73it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1043_tms_L_Fp_con_0001_mni.nii


  1%|▏         | 10/798 [00:01<02:24,  5.44it/s]

...

 98%|█████████▊| 785/798 [02:49<00:02,  4.77it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2064_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2066_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2069_tms_R_preSMA_con_0001_mni.nii


 99%|█████████▉| 794/798 [02:50<00:00,  6.16it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2088_tms_R_preSMA_con_0001_mni.nii


100%|██████████| 798/798 [02:50<00:00,  4.67it/s]


In [9]:
# individual roi 14mm - 10mm:
roi_dir = base_dir + "/3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Nov_3/tms_spheres/"
    
tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site'].replace('_', '-')
    subject = complete_df.iloc[i]['subject']
    
    mask_file = glob(roi_dir + site + "/" + str(subject) + "_14-10mm.nii.gz")
    
    if mask_file:
        try:
            nifti_masker = maskers.NiftiMasker(mask_file[0]).fit()
            tms_site_response.append(np.mean(nifti_masker.transform(image_name)))
        except:
            tms_site_response.append(np.nan)
            print("error occurs for mask: ", mask_file[0])
    else:
        tms_site_response.append(np.nan)
        print("mask not found: ", image_name)

complete_df['tms_site_response_individual_14-10mm'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)


  0%|          | 3/798 [00:00<03:01,  4.37it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1037_tms_L_Fp_con_0001_mni.nii


  1%|          | 7/798 [00:01<02:47,  4.71it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1043_tms_L_Fp_con_0001_mni.nii


  1%|▏         | 10/798 [00:01<02:34,  5.11it/s]

...

 98%|█████████▊| 785/798 [02:47<00:02,  4.54it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2064_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2066_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2069_tms_R_preSMA_con_0001_mni.nii


 99%|█████████▉| 792/798 [02:48<00:01,  4.77it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2088_tms_R_preSMA_con_0001_mni.nii


100%|██████████| 798/798 [02:49<00:00,  4.70it/s]


In [10]:
# individual roi 10mm - 6mm:
roi_dir = base_dir + "/3_Data_Working/CausalConnectome/post_processing/tmsfmri/William/Nov_3/tms_spheres/"
    
tms_site_response = []
for i in trange(len(complete_df)):
    image_name = complete_df.iloc[i]['inputfile']
    site = complete_df.iloc[i]['site'].replace('_', '-')
    subject = complete_df.iloc[i]['subject']
    
    mask_file = glob(roi_dir + site + "/" + str(subject) + "_10-6mm.nii.gz")
    
    if mask_file:
        try:
            nifti_masker = maskers.NiftiMasker(mask_file[0]).fit()
            tms_site_response.append(np.mean(nifti_masker.transform(image_name)))
        except:
            tms_site_response.append(np.nan)
            print("error occurs for mask: ", mask_file[0])
    else:
        tms_site_response.append(np.nan)
        print("mask not found: ", image_name)

complete_df['tms_site_response_individual_10-6mm'] = tms_site_response
# complete_df.to_csv(out_dir + out_file_name, index=False)

  0%|          | 3/798 [00:00<02:46,  4.77it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1037_tms_L_Fp_con_0001_mni.nii


  1%|          | 7/798 [00:01<02:37,  5.01it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/L_Fp/NTHC_2ndlevel/CausCon_NTHC1043_tms_L_Fp_con_0001_mni.nii


  1%|▏         | 10/798 [00:01<02:27,  5.33it/s]

...

 98%|█████████▊| 785/798 [06:15<00:02,  4.75it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2064_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2066_tms_R_preSMA_con_0001_mni.nii
mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2069_tms_R_preSMA_con_0001_mni.nii


 99%|█████████▉| 792/798 [06:16<00:01,  4.94it/s]

mask not found:  /Users/jingjiang/Library/CloudStorage/GoogleDrive-michaelniki1988@gmail.com/Shared drives/Jing_Drive/3_Data_Working/CausalConnectome/post_processing/tmsfmri/group_analysis_HRF_all/R_preSMA/TEHC_2ndlevel/CausCon_TEHC2088_tms_R_preSMA_con_0001_mni.nii


100%|██████████| 798/798 [06:17<00:00,  2.12it/s]


In [12]:
# complete_df.to_csv(out_dir + out_file_name, index=False)
complete_df

,inputfile,subject,site,group,tms_site_response_standard,tms_site_response_individual_6mm,tms_site_response_individual_10mm,tms_site_response_individual_14mm,tms_site_response_individual_14-10mm,tms_site_response_individual_10-6mm
0,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1001,L_Fp,NTHC,0.287125,0.439684,0.456288,0.395807,0.354767,0.462649
1,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1003,L_Fp,NTHC,-1.260076,1.178456,0.916780,0.524276,0.390585,0.894974
2,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1036,L_Fp,NTHC,-1.618781,-1.208849,-1.281960,-1.378883,-1.432572,-1.302709
3,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1037,L_Fp,NTHC,-0.337557,NaN,NaN,NaN,NaN,NaN
4,/Users/jingjiang/Library/CloudStorage/GoogleDr...,1038,L_Fp,NTHC,-0.523261,0.030765,0.065123,-0.066875,-0.118009,0.069139
...,...,...,...,...,...,...,...,...,...,...
793,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2090,R_preSMA,TEHC,0.214189,-0.228209,-0.128205,0.078518,0.208812,-0.092598
794,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2093,R_preSMA,TEHC,0.155220,0.220501,0.183733,0.107853,0.057474,0.168976
795,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2102,R_preSMA,TEHC,-0.534299,-1.183636,-0.868778,-0.557893,-0.361947,-0.756670
796,/Users/jingjiang/Library/CloudStorage/GoogleDr...,2105,R_preSMA,TEHC,-0.217289,-1.108896,-0.575760,-0.114780,0.170221,-0.383482
